In [58]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import numpy as np
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

In [115]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Basic CNN:
        # Convolution -> Relu -> Linear transformation -> Relu ->  Output (10 features)
        self.conv = nn.Conv2d(1, 1, 3)
        self.linear = nn.Linear(26 * 26, 10)
    
    def forward(self, x):
        # Define the forward pass here
        x = F.relu(self.conv(x))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.linear(x))
        return x
        
    def num_flat_features(self, x):
        # Image dimensionality
        # All dimensions except batch dimension
        # Batch dimension being # of inputs
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
net = Net()
print(net)

Net(
  (conv): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1))
  (linear): Linear(in_features=676, out_features=10, bias=True)
)


In [72]:
for p in list(net.parameters()):
    print(p.size())

torch.Size([1, 1, 3, 3])
torch.Size([1])
torch.Size([10, 900])
torch.Size([10])


In [97]:
# Loading up FashionMNIST dataset into train and test sets

# Transform images into tensors,
# With normalized pixel values in range [-1, 1]
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(0.5, 0.5)]
)

trainset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform = transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 4)
testset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform = transform)
testloader = torch.utils.data.DataLoader(testset, batch_size = 4)
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress',
           'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot')

In [116]:
import torch.optim as optim
# Optimizer handles updating weights

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
# 1. Zero the gradients for the next pass (PyTorch accumalates these gradients)
# 2. Make a forward pass through the network to get predictions
# 3. Get losses from our loss function 
# 4. Backpropagation calculates gradients of the loss w.r.t. our learnable parameters
# 5. Adjust the weights by Stochastic Gradient Descent

# Train the net over 4 epochs
num_epochs = 4
for repeat in range(num_epochs):
    for i, data in enumerate(trainloader, 0):
        in_data, labels = data
        # Forward pass
        output = net(in_data)
        
        # Zero gradients and calculate loss
        optimizer.zero_grad()

        loss = criterion(output, labels)
        
        # Backpropagate and adjust weights
        loss.backward()
        optimizer.step()